In [ ]:
# Importar as bibliotecas e funções necessárias para o case

import pandas as pd # type: ignore
import numpy as np # type: ignore
import sklearn as skl # type: ignore
import matplotlib.pyplot as plt # type: ignore
import plotly as plty # type: ignore
import plotly.express as px # type: ignore
import pysal as ps # type: ignore
import geobr as gbr # type: ignore
import geopandas as gpd # type: ignore
import sidrapy # type: ignore
import seaborn as sns # type: ignore
from shapely.geometry import Point # type: ignore
from libpysal.weights.contiguity import Queen # type: ignore
from esda.moran import Moran # type: ignore
from splot.esda import moran_scatterplot # type: ignore
from scipy.stats import spearmanr # type: ignore
from scipy.stats import gmean # type: ignore

## BASE DE DADOS

In [ ]:
# Carregar as duas bases de dados: POIs e faturamento

df_poi = pd.read_parquet("pontos_interesse_geograficos.parquet")
df_fat = pd.read_parquet("unidades_faturamento.parquet")

# Adicionar a geometria de cada linha para localização geográfica

df_poi['geometry'] = df_poi.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
df_fat['geometry'] = df_fat.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Transformar a base de dados de faturamento para o formato não pivotado (long format)

df_fat = df_fat.rename(columns=dict(zip(df_fat.columns[1:11], range(3, 13))))
unpivot_fat = df_fat.melt(id_vars=['id_unidade', 'geometry'], value_vars=range(3, 13), var_name='mês', value_name='faturamento')
unpivot_fat['id_unidade'] = unpivot_fat['id_unidade'].mod(500)
unpivot_fat = unpivot_fat.sort_values(by=['id_unidade', 'mês'])

# Transformar as bases de dados para GeoDataFrames

geo_poi = gpd.GeoDataFrame(df_poi.loc[:100000], crs=4326)
geo_fat = gpd.GeoDataFrame(df_fat, crs=4326)
geo_unpivot = gpd.GeoDataFrame(unpivot_fat, crs=4326)

# Carregar a base de dados dos municipios de SP

municipios = gbr.read_municipality(code_muni='SP')
municipios = municipios.to_crs(epsg=5880)

# Calcular o centroide dos municipios de SP (útil para os plots)

centroides = municipios.geometry.centroid
centroides = centroides.to_crs(epsg=4326)
centro_lat = centroides.y.mean()
centro_lon = centroides.x.mean()
municipios = municipios.to_crs(epsg=4326)

# Carregar a base de dados da população do estado de SP

populacao = sidrapy.get_table(table_code='4709', territorial_level='6', ibge_territorial_code='all', variable='93')
populacao_sp = populacao[populacao['D1C'].str.startswith('35')]
populacao_sp.loc[:, 'D1N'] = populacao_sp['D1N'].str.replace(' - SP', '', regex=False)
populacao_sp = populacao_sp[['D1N', 'V']]
populacao_sp.columns = ['name_muni', 'populacao']

## QUESTÃO 1

In [ ]:
# Cruzar os dados de geolocalização para determinar o municipio de cada POI

loc_poi = gpd.sjoin(geo_poi, municipios, how='inner', predicate='within')
loc_poi[['latitude', 'longitude', 'tipo_negocio', 'name_muni']]

In [ ]:
# Contar a frequência das categorias de negócio e mapear os potenciais concorrentes

categorias = pd.DataFrame(columns=['count', 'concorrencia'], index=df_poi['tipo_negocio'].value_counts().index)
categorias['count'] = df_poi['tipo_negocio'].value_counts()

# Classificar os potenciais concorrentes como -1

for tipo in ['restaurante', 'supermercado', 'padaria', 'shopping', 'farmacia']:
    categorias.loc[tipo, 'concorrencia'] = -1

# Classificar os não-concorrentes como 0

for tipo in ['comercio', 'negocio', 'prestador', 'beleza', 'igreja',
             'medico', 'outros', 'oficina', 'evento', 'escritório',
             'dentista', 'transporte', 'fornecedor', 'petshop',
             'hospedagem', 'concessionaria', 'amenidade', 'clinica',
             'publico', 'academia', 'combustivel', 'organizacao',
             'moradia', 'estacionamento', 'esporte', 'veterinario',
             'caixa', 'banco', 'hospital', 'saude', 'distribuidor',
             'fabricante', 'laboratorio', 'policia', 'servicopublico',
             'fazenda', 'autoescola', 'eventos', 'correio', 'cartorio',
             'bombeiro', 'aeroporto', 'exercito', 'heliporto']:
    categorias.loc[tipo, 'concorrencia'] = 0

# Classificar os potenciais colaboradores como 1 (POIs que possam estimular as vendas de um varejo de doces)

for tipo in ['escola', 'universidade', 'cinema', 'clube', 'praia publica']:
    categorias.loc[tipo, 'concorrencia'] = 1

## QUESTÃO 2

In [ ]:
# Contar os POIs por município

poi_contagem = loc_poi['name_muni'].value_counts().reset_index()
poi_contagem.columns = ['name_muni', 'count']
poi_contagem = municipios.merge(poi_contagem, on='name_muni', how='left')
poi_contagem['count'] = poi_contagem['count'].fillna(0)

# Plotar os POIs e a distribuição de POIs por município

figure = px.scatter_map(loc_poi, lat='latitude', lon='longitude', zoom=5, hover_data='tipo_negocio', title=f'POIs')
figure.show()
figure = px.choropleth_map(poi_contagem, geojson=poi_contagem.geometry,  locations=poi_contagem.index, zoom=5,
                           color='count', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                           hover_name='name_muni', title='Quantidade total de POIs por município')
figure.show()

## QUESTÃO 3

In [ ]:
# Plotar a distribuição dos POIs das top 5 categorias por município

top_categorias = df_poi['tipo_negocio'].value_counts().index[:5]

for tipo in top_categorias:
    loc_top = loc_poi[loc_poi['tipo_negocio'] == tipo]
    top_contagem = loc_top['name_muni'].value_counts().reset_index()
    top_contagem.columns = ['name_muni', 'count']
    top_contagem = municipios.merge(top_contagem, on='name_muni', how='left')
    top_contagem['count'] = top_contagem['count'].fillna(0)
    figure = px.choropleth_map(top_contagem, geojson=top_contagem.geometry,  locations=top_contagem.index,
                               zoom=5, color='count', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                               title=f'Quantidade total de POIs de {tipo} por município')
    figure.show()

## QUESTÃO 4

In [ ]:
# Plotar a distribuição dos POIs concorrentes por município

loc_concorrente = loc_poi[loc_poi['tipo_negocio'].isin(categorias[categorias['concorrencia'] == -1].index)]
concorrente_contagem = loc_concorrente['name_muni'].value_counts().reset_index()
concorrente_contagem.columns = ['name_muni', 'count']
concorrente_contagem = municipios.merge(concorrente_contagem, on='name_muni', how='left')
concorrente_contagem['count'] = concorrente_contagem['count'].fillna(0)
figure = px.choropleth_map(concorrente_contagem, geojson=concorrente_contagem.geometry,  locations=concorrente_contagem.index,
                           zoom=5, color='count', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                           title=f'Quantidade total de POIs concorrentes por município', hover_name='name_muni')
figure.show()

## QUESTÃO 5

In [ ]:
# Criar a matriz de contiguidade (vizinhança) dos municipios (menos Ilhabela, por ser uma ilha)

weights = Queen.from_dataframe(municipios.drop(232), use_index=False)

# Calcular o coeficiente I de Moran global da quantidade de POIs por município

y = poi_contagem.drop(232)['count'].copy()
weights.transform = 'o'
moran = Moran(y, weights, permutations=9999)
print(f'Moran I Global POIs por municipio: {moran.I}')

# Plotar a respectiva regressão valor x lag

moran_scatterplot(moran, aspect_equal=True)
plt.show()

# Calcular o coeficiente I de Moran global da quantidade de POIs das top 5 categorias por município

for tipo in top_categorias:
    loc_top = loc_poi[loc_poi['tipo_negocio'] == tipo]
    top_contagem = loc_top['name_muni'].value_counts().reset_index()
    top_contagem.columns = ['name_muni', 'count']
    top_contagem = municipios.merge(top_contagem, on='name_muni', how='left')
    top_contagem['count'] = top_contagem['count'].fillna(0)
    y_top = top_contagem.drop(232)['count'].copy()
    weights.transform = 'o'
    moran = Moran(y_top, weights, permutations=9999)
    print(f'Moran I Global POIs de {tipo} por municipio: {moran.I}')

    # Plotar a respectiva regressão valor x lag

    moran_scatterplot(moran, aspect_equal=True)
    plt.show()

# Calcular o coeficiente I de Moran global da quantidade de POIs concorrentes por município

y_concorrente = concorrente_contagem.drop(232)['count'].copy()
weights.transform = 'o'
moran = Moran(y_concorrente, weights, permutations=9999)
print(f'Moran I Global POIs concorrentes por municipio: {moran.I}')

# Plotar a respectiva regressão valor x lag

moran_scatterplot(moran, aspect_equal=True)
plt.show()

## QUESTÃO 6

In [ ]:
# Calcular e plotar mediana, media e soma dos faturamentos ao longo dos períodos

geo_fat['mediana_fat'] = geo_fat[range(3,13)].median(axis=1)
geo_fat['media_fat'] = geo_fat[range(3,13)].mean(axis=1)
geo_fat['fat_total'] = geo_fat[range(3,13)].sum(axis=1)
figure = px.scatter_map(geo_fat, lat='latitude', lon='longitude', hover_data='mediana_fat', zoom=5,
                        color='mediana_fat', color_continuous_scale='Blues',
                        title='Mediana dos faturamentos por unidade')
figure.show()

## QUESTÃO 7

In [ ]:
# Calcular e plotar os coeficientes de correlação de Spearman (faturamento x tempo) do faturamento em cada unidade (tendências de alta ou de baixa)

spearman_unidade = geo_unpivot.groupby('id_unidade').apply(lambda group: spearmanr(group['mês'], group['faturamento']))
spearman_unidade = spearman_unidade.apply(lambda x: pd.Series(x, index=['rho', 'pvalor']))
spearman_unidade = spearman_unidade.reset_index()
spearman_unidade = geo_fat.merge(spearman_unidade, on='id_unidade', how='left')
spearman_unidade

figure = px.scatter_map(spearman_unidade, lat='latitude', lon='longitude', zoom=5, color='rho',
                        color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                        title='Correlação de Spearman faturamento x tempo por unidade')
figure.show()

In [ ]:
# Calcular e plotar a média dos faturamentos por município em cada período

loc_unpivot = gpd.sjoin(geo_unpivot, municipios, how='inner', predicate='within')
fat_periodo = loc_unpivot.groupby(['name_muni', 'mês'])['faturamento'].mean()
fat_periodo = fat_periodo.reset_index()
fat_periodo = municipios.merge(fat_periodo, on='name_muni', how='left')
fat_periodo = fat_periodo.dropna(subset=['mês'])

figure = px.choropleth_map(fat_periodo, geojson=fat_periodo.geometry,  locations=fat_periodo.index, zoom=5,
                           color='faturamento', center={"lat": centro_lat, "lon": centro_lon}, animation_frame='mês',
                           title='Faturamento total médio por município em cada período')
figure.show(renderer='browser')

In [ ]:
# Calcular e plotar os coeficientes de correlação de Spearman (faturamento x tempo) do faturamento total em cada municipio (tendências de alta ou de baixa)

spearman_municipio = fat_periodo.groupby('name_muni').apply(lambda group: spearmanr(group['mês'], group['faturamento']))
spearman_municipio = spearman_municipio.apply(lambda x: pd.Series(x, index=['rho', 'pvalor']))
spearman_municipio = spearman_municipio.reset_index()
spearman_municipio = municipios.merge(spearman_municipio, on='name_muni', how='left')
spearman_municipio['rho'] = spearman_municipio['rho'].fillna(0)

figure = px.choropleth_map(spearman_municipio, geojson=spearman_municipio.geometry,  locations=spearman_municipio.index, zoom=5,
                           color='rho', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon}, hover_name='name_muni',
                           title='Correlação de Spearman faturamento x tempo por município')
figure.show()

## QUESTÃO 8

In [ ]:
# Plotar a média por município dos faturamentos totais das unidades

loc_fat = gpd.sjoin(geo_fat, municipios, how='inner', predicate='within')
fat_municipio = loc_fat.groupby('name_muni')[['mediana_fat', 'media_fat', 'fat_total']].mean()
fat_municipio = municipios.merge(fat_municipio, on='name_muni', how='left')
fat_municipio = fat_municipio.fillna(0)

figure = px.choropleth_map(fat_municipio, geojson=fat_municipio.geometry,  locations=fat_municipio.index, zoom=5,
                           color='fat_total', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                           hover_name='name_muni', title='Faturamento total médio por município')
figure.show()

## QUESTÃO 9

In [ ]:
# Plotar a média por 100 mil habitantes dos faturamentos totais das unidades

fat_municipio['soma_fat_total'] = fat_municipio['name_muni'].map(loc_fat.groupby('name_muni')['fat_total'].sum())
fat_municipio['populacao'] = pd.to_numeric(populacao_sp['populacao']).values
fat_municipio['fat_hab'] = fat_municipio['soma_fat_total']/fat_municipio['populacao'] * 100000
fat_municipio = fat_municipio.fillna(0)

figure = px.choropleth_map(fat_municipio, geojson=fat_municipio.geometry,  locations=fat_municipio.index, zoom=5,
                           color='fat_hab', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                           hover_name='name_muni', title='Faturamento total médio por 100 mil habitantes')
figure.show()

## QUESTÃO 10

In [ ]:
# Calcular o coeficiente I de Moran global do faturamento total por município

y = fat_municipio.drop(232)['soma_fat_total'].copy()
weights.transform = 'o'
moran = Moran(y, weights, permutations=9999)
print(f'Moran I Global faturamento total por municipio: {moran.I}')

# Plotar a respectiva regressão valor x lag

moran_scatterplot(moran, aspect_equal=True)
plt.show()

# Plotar o faturamento total por município

figure = px.choropleth_map(fat_municipio, geojson=fat_municipio.geometry,  locations=fat_municipio.index, zoom=5,
                           color='soma_fat_total', color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                           hover_name='name_muni', title='Faturamento total por município')
figure.show()

## QUESTÃO 11

Primeiramente, sintetizo aqui as conclusões mais diretas extraídas das questões anteriores:

- A região metropolitana de São Paulo contém a maior concentração de POIs do estado, especialmente a cidade de São Paulo. Outros municípios desenvolvidos e grandes em termos de população também se destacam, como Campinas, Ribeirão Preto, São José dos Campos, Piracicaba e Sorocaba. O mesmo vale para as top 5 categorias de POIs.

- Restaurantes e supermercados se enquadram como potenciais concorrentes para uma rede varejista do setor alimentício e estão entre os POIs com maior frequência no estado, de forma que a presença destas pode estar relacionada com o faturamento da rede.

- Os coeficientes de Moran I positivos e acima de 0.3 indicam uma correlação positiva entre o número de POIs e a distribuição espacial, isto é, há uma similaridade (não muito alta) entre a número de POIs em um município e em seus vizinhos. Destaca-se novamente a cidade de São Paulo (ponto no extremo direito inferior do Moran scatterplot) como um hot-spot com muito mais POIs do que seus vizinhos (região metropolitana), apesar dos vizinhos terem valores relativamente altos também. Em especial, o setor de beleza e os POIs concorrentes obtiveram maiores coeficientes de correlação, implicando uma previsibilidade maior na localização destes POIs.

- A localização das unidades da rede varejista segue uma distribuição parecida com os POIs em geral, mas faturamentos de cada unidade são menos previsiveis, com unidades de alto faturamento em municípios distantes da região metropolitana de São Paulo e com menos habitantes, e isso reflete na média de faturamento por município. Exemplos são os municípios de Itápolis e Artur Nogueira, com unidades de faturamento extremamente alto, o que pode se dever ao efeito de 'canibalização', no qual filiais mais 'isoladas' tem mais chances de sucesso por não competirem com outras filiais da mesma rede (em seguida analisaremos isso). O mesmo ocorre com a média de faturamento por 100 mil habitantes, destacando-se os municípios de Iepê e Jambeiro.

- O faturamento total por município não sofre do mesmo efeito devido a alta concentração de unidades da rede em São Paulo e outros municípios grandes, mas ainda assim o coeficiente de Moran I do faturamento total é inferior a 0.3.

- Por fim, analisando as variações, mediante correlação de Spearman, dos faturamentos por unidade e por município ao longo do tempo, percebemos tendências de alta e de baixa em municípios variados: cidades grandes próximas de São Paulo mostram perspectiva de crescimento (apesar da cidade de São Paulo não estar crescendo muito), assim como pequenos municípios distantes da região central do estado; por outro lado, alguns municípios pequenos (em população), com as mais diversas localizações, como Ibiúna e Guararapes, apresentam uma tendência de baixa.

Vale destacar que muitos destes resultados eram esperados de certa forma, sabendo-se a distribuição de renda do estado de São Paulo.

### Correlação espacial das tendências de alta e baixa

In [ ]:
# Calcular o coeficiente I de Moran global do coeficiente de Spearman por município (tendência de alta e baixa)

y_spearman = spearman_municipio.drop(232)['rho'].copy()
weights.transform = 'o'
moran = Moran(y_spearman, weights, permutations=9999)
print(f'Moran I Global coeficiente de Spearman por município: {moran.I}')

# Plotar a respectiva regressão valor x lag

moran_scatterplot(moran, aspect_equal=True)
plt.show()

### Variação média e projeção de faturamento

In [ ]:
# Calcular e plotar a variação percentual média do faturamento de cada unidade

geo_fat['variacao_media'] = (1 + geo_fat[range(3, 13)].pct_change(axis=1)).iloc[:, 1:].apply(gmean, axis=1) - 1

figure = px.scatter_map(geo_fat, lat='latitude', lon='longitude', zoom=5, color='variacao_media',
                        color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                        title='Variação percentual média do faturamento por unidade')
figure.show()

# Calcular e plotar a projeção do faturamento por unidade do próximo mês

geo_fat['previsao_mês'] = (1 + np.abs(geo_fat['variacao_media'])*spearman_unidade['rho'])*geo_fat[12]

figure = px.scatter_map(geo_fat, lat='latitude', lon='longitude', zoom=5, color='previsao_mês',
                        color_continuous_scale='Blues', center={"lat": centro_lat, "lon": centro_lon},
                        title='Projeção do faturamento por unidade para o próximo mês')
figure.show()

### Efeito 'canibalização' e correlação com tipo de POI

In [ ]:
# Calcular a correlação entre o faturamento e a quantidade de unidades próximas para raios variando entre 1km e 700km (canibalização)

dgeo_poi = geo_poi.to_crs(epsg=32723)
dgeo_fat = geo_fat.to_crs(epsg=32723)

canibalismo = pd.Series()
raios = range(1, 700, 5)

for raio in raios:
    distancia_unidades = dgeo_fat.geometry.apply(lambda geom: dgeo_fat.geometry.distance(geom))
    unidades_proximas = pd.DataFrame(((distancia_unidades < raio*1000) & (distancia_unidades > 0)).sum(axis=1))
    unidades_proximas['faturamento'] = geo_fat['fat_total']
    canibalismo[f'{raio}km'] = unidades_proximas.corr()[0]['faturamento']

plt.plot(raios, canibalismo.values)
plt.title('Correlação entre o faturamento e a quantidade de unidades próximas (a um raio de x km)')
plt.show()

In [ ]:
# Calcular a correlação entre o faturamento e a quantidade de POIs de cada tipo próximos para raios variando entre 0.5km e 20km

correlacao_poi = pd.DataFrame()
raios = [500, 1000, 2000, 5000, 10000, 20000]

for raio in raios:
    negocios_proximos = {}
    for name, group in dgeo_poi.groupby(by='tipo_negocio'):
        negocios_proximos[name] = (group.geometry.apply(lambda geom: dgeo_fat.geometry.distance(geom)) < raio).sum(axis=0)
    negocios_proximos = pd.DataFrame(negocios_proximos)
    negocios_proximos['faturamento'] = geo_fat['fat_total']
    correlacao_poi[f'{raio/1000}km'] = negocios_proximos.corr()['faturamento']

correlacao_poi = correlacao_poi.drop('faturamento')

# Plotar apenas as correlações com maior valor absoluto para cada raio

for raio in correlacao_poi.columns:
    sns.barplot(correlacao_poi[raio].sort_values(key=abs, ascending=False).head())
    plt.title(f'Correlação entre faturamento e quantidade de POIs a um raio de {raio}')
    plt.show()

### Conclusões finais

Agora, algumas conclusões finais um pouco mais aprofundadas com base nas análises acima:

- Como esperado anteriormente, não existe uma correlação relevante entre as tendências de alta e baixa no faturamento por município e a distribuição espacial, indicando uma dificuldade em distinguir regiões do estado com maiores perspectivas de crescimento no faturamento.

- As projeções de faturamento (com base na variação média e no coeficiente de Spearman) de cada unidade para o próximo mês apontam que em média as unidades estão com uma tendência de crescimento e que as unidades mais em alta estão distribuidas pelo estado (e não concentradas em regiões específicas). Contudo, essa projeção ainda é pouco confiável e portanto não fornece conclusões mais aprofundadas do que isso.

- A análise do efeito de 'canibalização' traz alguns ponto interessantes, apesar de estatisticamente pouco relevantes (as correlações ainda são muito baixas em valor absoluto): para um raio em torno de 100km, a presença de outras unidades parece impulsionar minimamente os faturamentos, enquanto que para raios entre 500km e 600km o efeito é reverso e em menor intensidade. A correlação do faturamento com os tipos de POIs também aponta algumas conclusões a priori: potencialmente, a presença de alguns POIs considerados anteriormente 'concorrentes' como supermercados e farmácias, nas proximidades das unidades está correlacionado a um maior faturamento (correlação, não causalidade!). Setores de saúde, fazenda, fabricante e clube a uma distância de no máximo 2km supostamente impulsionam o faturamento, enquanto a presença de laboratorios, estacionamentos, eventos e transportes na proximidade, ou até mesmo exército a uma distância de 20km, podem estar relacionados a uma redução no faturamento das unidades.